# UPenn Flywheel Data Transfer to ASC FMRISrv 

This notebook was shared by Dr Nicole Cooper from CNLab referenced this notebook for Flywheel MURI scan downloads as an example... The same should work for CNLab & AHALab Flywheel projects.

+ 02/21/2020 - [José Carreras-Tartak](mailto:jcarreras@falklab.org) original author
+ 04/28/2021 - [Etienne Jacquot](mailto:etienne.jacquot@asc.upenn.edu) revisited


## *Getting Started w/ [UPenn Flywheel](https://upenn.flywheel.io/) Python-SDK*:

The AHA lab does not have a project on Flywheel so maybe not all the steps are exact yet. This eventually will be in place though. For now let us try based on a specific sessionID

- Please navigate here for access via Pennkey: https://upenn.flywheel.io/
- you need an **api key**, be careful with this secret


In [1]:
import flywheel
import tarfile
import os
import time
import zipfile
from zipfile import ZipFile

import configparser

### Create you Flywheel API secret config file 


- If the cell below returns True, you have a .config file with an api key of some kind. If it returns false OR if your api key needs to be changed, you can run the cell below that to create your apikey or manually navigate and create your [configs/config.ini](./configs/config.ini) (_we add *.ini to the .gitignore_)



_____

```python
config['UPENN-FLYWHEEL'] = {'apikey':your_api_key}  # <-- define your api key

with open(home_dir + '/configs/config.ini', 'w') as configFile: # <-- write to file!
    config.write(configFile)

```

_______

### Read Flywheel API secret into Python w/ ConfigParser

- You must login and navigate to https://upenn.flywheel.io/#/profile, this has your API key 


In [2]:
# Check if the API Key exists. If the output is false
home_dir = "/home/aresnick@asc.upenn.edu"

config = configparser.ConfigParser()
config.read(home_dir + "/configs/config.ini")
config.has_option('UPENN-FLYWHEEL', 'apikey')

True

In [ ]:
# specify your home directory where the config file should be saves

your_api_key = "upenn.flywheel.io:YOUR_API_KEY"

config = configparser.ConfigParser()
config['UPENN-FLYWHEEL'] = {'apikey':your_api_key}
with open(home_dir + '/configs/config.ini', 'w') as configFile:
    config.write(configFile)

In [3]:
# add UPenn Flywheel api key to your config.ini
fw_cred = {}
config = configparser.ConfigParser()

config.read(home_dir + '/configs/config.ini') 
for item,value in config['UPENN-FLYWHEEL'].items():
    fw_cred[item]=value

In [4]:
# read your API key
api = fw_cred['apikey']

### Confirm your access to Flywheel via python SDK

- The `fw.get_current_user()` command is a quick way to ensure you have established a secure connection to UPenn Flywheel

In [5]:
# Create client using your API key
fw = flywheel.Client(api)

In [19]:
# print your flywheel information & confirm it works as expected
self = fw.get_current_user()
print('UPenn Flywheel User: %s %s (%s)' % 
      (self.firstname, self.lastname, self.email))

UPenn Flywheel User: Anthony Resnick (aresni@upenn.edu)


_______

## Proceed by Navigating to Flywheel, you'll notice the URL always has respective identifiers

In this example, our notebook tests for a known session ID associated w/ Dr Lydon-Staley AHA Lab:

- https://upenn.flywheel.io/#/projects/5ba2913fe849c300150d02ed/sessions/6088730ee6de2e3066bd7249
    - where the session ID is in the URL --> `6088730ee6de2e3066bd7249`



### Set your Flywheel Project Container & Corresponding Local Out Project


In [7]:
# replace with name of Flywheel project container (i.e. "geoscan")
# in_project = "geoscan"
in_project = 'geoscan'

# replace with output project folder name in fMRI server (i.e. "geoscanR01")
# out_project = "GS"
out_project = "geoscan_R01"

### Set your session specific ID & corresponding out ID

- not sure why the `opID` is entirely needed here... TBD

In [158]:
## MODIFY BELOW
# replace with ppt ID as listed on Flywheel (e.g. for geoscan, typically "gsXXX")
#ipID = "gs004"
ipID = "gs032_t2"

# replace with ppt ID as it will be stored in the server (i.e. "GSXXX")
#opID = "GS004"
opID = 'GS032_t2' # <--- I think this could be whatever, so long as this is unique on the FMRI host

### Verify that output directory in the server is accurate

- You may need to create this directory ahead of time...

In [159]:
outpath = '/fmriDataRaw/fmri_data_raw/{PROJECT}'.format(PROJECT=out_project)

os.listdir(outpath)

['GS004-T2',
 'GS005',
 'GS008',
 'GS016',
 'GS017_t2',
 'GS017_t3',
 'GS020',
 'GS022_t2',
 'GS022_t3',
 'GS024_t2',
 'GS024_t3',
 'GS025_t2',
 'GS025_t3',
 'GS026',
 'GS028_t2',
 'GS028_t3',
 'GS031_t2',
 'GS031_t3']

________

## Proceed with looking up your subject data & downloading Dicom tarball

NOTE!!

* Location for DICOMS on the server IS:

    - `/fmriDataRaw/fmri_data_raw/{PROJECT}/`

e.g. untar the appropriate folder to e.g. `/fmriDataRaw/fmri_data_raw/{PROJECT}/`


### Flywheel uses `Group / Project / Subject / Session` to identify scan ... 

- the **group** is `falklab`

- the **project** is `bbprime` *(fw://unknown/Unsorted)*

- the **subject** is `bpp00` *(probably a default for the unsorted group)*

- the **session** is `CAMRIS^Falk`

#### Thus our lookup string is --> `'falklab/bbprime/bpp00/CAMRIS^Falk'` 

In [160]:
#group_label = 'falklab'
group_label = 'falklab'

#project_label = 'bbprime'
project_label = in_project # <-- values are set early on in the notebook... maybe that isn't helpful though?

#subject_label = 'bpp00'
subject_label = ipID # <-- values are set early on in the notebook... maybe that isn't helpful though?

session_label = 'CAMRIS^Falk'

######################################################

lookup_string = '{}/{}/{}/{}'.format(group_label,project_label,subject_label,session_label)
lookup_string

'falklab/geoscan/gs032_t2/CAMRIS^Falk'

### Proceed with looking up the known session in the *Unsorted* project

Create `session` object to lookup session of interest, you want to then confirm metadata is accurate!

- For a helpful video overview on finding your data on Flywheel w/ Python-SDK, I strongly encourage you to visit here:
https://docs.flywheel.io/hc/en-us/articles/360048440933-Webinar-Series-Finding-your-stuff-in-Flywheel-with-the-Python-SDK

*TODO --> CONTACT UPENN FLYWHEEL ADMIN TEAM TO FIGURE OUT LAB PROJECTS!*

In [161]:
#session = fw.lookup('{}'.format(lookup_string))
session = fw.lookup('{group}/{proj}/{pid}'.format(group=group_label,proj=in_project,pid=ipID))
session

{'age': None,
 'analyses': [],
 'code': 'gs032_t2',
 'cohort': None,
 'created': datetime.datetime(2021, 12, 10, 21, 42, 23, 445000, tzinfo=tzutc()),
 'ethnicity': None,
 'files': [],
 'firstname': None,
 'id': '61b3c9bf8022190b119f112d',
 'info': {},
 'info_exists': False,
 'label': 'gs032_t2',
 'lastname': None,
 'master_code': None,
 'modified': datetime.datetime(2021, 12, 10, 21, 42, 23, 445000, tzinfo=tzutc()),
 'notes': [],
 'parents': {'acquisition': None,
             'analysis': None,
             'group': 'falklab',
             'project': '5e2b4677048f71007908099d',
             'session': None,
             'subject': None},
 'permissions': [],
 'project': '5e2b4677048f71007908099d',
 'race': None,
 'revision': 1,
 'sex': None,
 'species': None,
 'strain': None,
 'tags': [],
 'type': None}

### Download the Flywheel Session tarball to FMRISrv

- Once we have the tar zip we can then extract our dicoms to the network


- *On running for Dr Lydon-Staley test subject, this tarball file is nearly 1GB*

#### What about the `./working_data` directory? 

*TODO --> Where does working data directory go? Is that just in the jupyterhub environment? does the tarball get deleted after or saved to the network in raw data?* 
*working data is still here! -AR*

In [162]:
!mkdir working_data

mkdir: cannot create directory ‘working_data’: File exists


In [163]:
fw.download_tar(session,'./working_data/{opID}.tar'.format(opID=opID))


{'file_cnt': 18,
 'size': 364023217,
 'ticket': '50f57c6f-bf95-4a41-bdab-c420261a5817'}

## Extract contents of Flywheel tar download:

In the following cells, you will:

1. Load tarball into jupyterhub notebook memory space

2. Set your dicom out directory and confirm permissions

3. Loop through tarball `.getmembers()` and then extract zipped dicoms

### Load into Memory:

In [164]:
f = open("working_data/{opID}.tar".format(opID=opID), 'rb') # <--- Flywheel download as Read Bytes
print ('Opening tar in memory as:',f,'\n')
tar_data = tarfile.open(fileobj=f, mode='r:') # <--- Unpack tar in memory

Opening tar in memory as: <_io.BufferedReader name='working_data/GS032_t2.tar'> 



### Set and Create your Out Directory:

- Jupyterhub does not respect secondary group permissions... so when I create a directory it's default to FMRISrvUser1@asc.upenn.edu instead of FMRISrvAHAUsers@asc.upenn.edu ... will manually correct

In [165]:
output_dicom_dir = '{outpath}/{opID}/'.format(outpath=outpath,opID=opID)
print(output_dicom_dir)

/fmriDataRaw/fmri_data_raw/geoscan_R01/GS032_t2/


In [166]:
# Create the directory if not exist
if not os.path.exists(os.path.dirname(output_dicom_dir)):
    try:
        print('makedirs --> {}'.format(output_dicom_dir))
        os.makedirs(os.path.dirname(output_dicom_dir))
    except:
        print('oops! failed to create --> {}'.format(output_dicom_dir))        

makedirs --> /fmriDataRaw/fmri_data_raw/geoscan_R01/GS032_t2/


## Confirm permissions for out directory

### Had to make the outdir permission 777 -R

- Secondary group permission is not respected in jhub so I had to manually change for my user created folder ... 

```bash
sudo chgrp fmrisrvahausers@asc.upenn.edu -R /AHAData/fmri_data_raw/
```

In [167]:
ls -la $output_dicom_dir

total 0
drwxrwxr-x. 1 aresnick@asc.upenn.edu fmrisrv1users@asc.upenn.edu 0 Jun 13 00:22 ./
drwxrwxr-x. 1 mbod@asc.upenn.edu     fmrisrv1users@asc.upenn.edu 0 Jun 13 00:22 ../


## EXTRACT YOUR TARBALL DICOM TO FMRISRV NETWORK STORAGE

In [168]:
for member in tar_data.getmembers():
    
    if 'dicom.zip' in member.name:       # <--- Only extract files with 'dicom.zip' 
        
        print('Extracting: {}\n'.format(member.name))
        
        tfile = tar_data.extractfile(member.name)
        dicom_zip = zipfile.ZipFile(tfile, mode='r')
        dicom_zip.extractall(output_dicom_dir)
tar_data.close()

Extracting: scitran/falklab/geoscan/gs032_t2/CAMRIS^Falk/localizer_multislice/1.3.12.2.1107.5.2.43.66044.2021121016341149480017035.0.0.0.dicom.zip

Extracting: scitran/falklab/geoscan/gs032_t2/CAMRIS^Falk/AAHead_Scout/1.3.12.2.1107.5.2.43.66044.202112101635131792317842.0.0.0.dicom.zip

Extracting: scitran/falklab/geoscan/gs032_t2/CAMRIS^Falk/AAHead_Scout_MPR_sag/1.3.12.2.1107.5.2.43.66044.2021121016351663816618885.0.0.0.dicom.zip

Extracting: scitran/falklab/geoscan/gs032_t2/CAMRIS^Falk/AAHead_Scout_MPR_cor/1.3.12.2.1107.5.2.43.66044.2021121016351663861118891.0.0.0.dicom.zip

Extracting: scitran/falklab/geoscan/gs032_t2/CAMRIS^Falk/AAHead_Scout_MPR_tra/1.3.12.2.1107.5.2.43.66044.2021121016351663884218895.0.0.0.dicom.zip

Extracting: scitran/falklab/geoscan/gs032_t2/CAMRIS^Falk/PhoenixZIPReport/1.3.12.2.1107.5.2.43.66044.30000021120923390393900000156.dicom.zip

Extracting: scitran/falklab/geoscan/gs032_t2/CAMRIS^Falk/BOLD_IMAGE_run01/1.3.12.2.1107.5.2.43.66044.202112101641469447219431.0

### You have now successfully downloaded the dicom data from Flywheel to ASC servers

- this goes to `/fmriDataRaw/fmri_data_raw/bbprime/BPP00/`

In [124]:
os.listdir('{}'.format(output_dicom_dir))


['1.3.12.2.1107.5.2.43.66044.2021112911041388531815681.0.0.0.dicom',
 '1.3.12.2.1107.5.2.43.66044.2021112911051530623416431.0.0.0.dicom',
 '1.3.12.2.1107.5.2.43.66044.2021112911051894533717531.0.0.0.dicom',
 '1.3.12.2.1107.5.2.43.66044.2021112911051894574117537.0.0.0.dicom',
 '1.3.12.2.1107.5.2.43.66044.2021112911051894597717541.0.0.0.dicom',
 '1.3.12.2.1107.5.2.43.66044.2021112911075572401018076.0.0.0.dicom',
 '1.3.12.2.1107.5.2.43.66044.2021112911115081901019205.0.0.0.dicom',
 '1.3.12.2.1107.5.2.43.66044.2021112911183849813537690.0.0.0.dicom',
 '1.3.12.2.1107.5.2.43.66044.2021112911190368805938255.0.0.0.dicom',
 '1.3.12.2.1107.5.2.43.66044.2021112911263612666656740.0.0.0.dicom',
 '1.3.12.2.1107.5.2.43.66044.202111291126576452957305.0.0.0.dicom',
 '1.3.12.2.1107.5.2.43.66044.2021112911340233718875790.0.0.0.dicom',
 '1.3.12.2.1107.5.2.43.66044.2021112911342331441676355.0.0.0.dicom',
 '1.3.12.2.1107.5.2.43.66044.202111291141346549894840.0.0.0.dicom',
 '1.3.12.2.1107.5.2.43.66044.2021112

## Delete .tar files?

If you would like to delete the .tar files, open the terminal, navigate to this directory, and enter the following code to recursively empty the .tar folder. You can either keep these or download them again as needed from flywheel. *BE CAREFUL* Make sure you are pointing to the correct directory! This will recursively delete every file in a directory!

```python
rm -r [path/to/DICOMS/working_data/]
```